In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/span-test/es_test_without_labels.csv
/kaggle/input/polyhope-dataset/en_dev.csv
/kaggle/input/polyhope-dataset/es_train.csv
/kaggle/input/polyhope-dataset/es_dev.csv
/kaggle/input/polyhope-dataset/en_train.csv
/kaggle/input/polyhope/en_dev.csv
/kaggle/input/polyhope/en_train.csv
/kaggle/input/test-phase/en_test_without_labels.csv
/kaggle/input/test-phase/es_test_without_labels.csv


In [7]:
!pip install pyspellchecker

In [8]:
!pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 72.3 MB/s eta 0:00:00:00:010:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow import keras
plt.style.use('ggplot')
import nltk
import string
import spacy
import re

In [3]:
df_eng= pd.read_csv('english data path')
df_eng.head()

,text,binary,multiclass
0,#USER# #USER# #USER# #USER# You expect a man t...,Not Hope,Not Hope
1,#USER# #USER# #USER# #USER# Tinubu is actually...,Not Hope,Not Hope
2,it'd be nice if missguided actually had stock ...,Hope,Sarcasm
3,#USER# Anyway love u bubbly i know i can count...,Hope,Generalized Hope
4,“you have a lot of people rooting for you whet...,Not Hope,Not Hope


In [4]:
df_esp = pd.read_csv('spanish data path')
df_esp.head()

,text,binary,multiclass
0,🎬Outer Banks🎬 (temporada 3) la verdad que teni...,Hope,Generalized Hope
1,-Después del poco éxito en el grupo de la Jet ...,Hope,Generalized Hope
2,Bárbara López yo te rezo y te hago un monument...,Hope,Generalized Hope
3,RT #USER# #USER# #USER# Ojalá nos ayuden a des...,Hope,Realistic Hope
4,#USER# No tengo las palabras justas. Pero rezo...,Hope,Realistic Hope


In [5]:
df_eng['binary'].value_counts()

binary
Not Hope    2807
Hope        2426
Name: count, dtype: int64

In [6]:
df_esp['binary'].value_counts()

binary
Not Hope    5927
Hope        5316
Name: count, dtype: int64

In [7]:
df_eng['multiclass'].value_counts()

multiclass
Not Hope            2245
Generalized Hope    1284
Sarcasm              692
Realistic Hope       540
Unrealistic Hope     472
Name: count, dtype: int64

In [8]:
df_esp['multiclass'].value_counts()

multiclass
Not Hope            5383
Generalized Hope    2754
Unrealistic Hope    1300
Realistic Hope      1113
Sarcasm              693
Name: count, dtype: int64

In [9]:
dev_eng = pd.read_csv('validate english data path')
dev_eng.head()

,text,binary,multiclass
0,And you’d come to the obvious logical solution...,Hope,Generalized Hope
1,#USER# I’m hoping his type helps quiet our wor...,Hope,Realistic Hope
2,I aspire to be his sugar mommy...I know chance...,Hope,Unrealistic Hope
3,also that jacket he wore yesterday i���m gonna...,Hope,Sarcasm
4,How can they visually express themselves witho...,Not Hope,Sarcasm


In [10]:
dev_eng['binary'].value_counts()

binary
Not Hope    1003
Hope         899
Name: count, dtype: int64

In [14]:
dev_eng['multiclass'].value_counts()

multiclass
Not Hope            816
Generalized Hope    467
Sarcasm             252
Realistic Hope      196
Unrealistic Hope    171
Name: count, dtype: int64

In [11]:
dev_esp = pd.read_csv('validate spanish data path')
dev_esp.head()

,text,binary,multiclass
0,#USER# #USER# Ole ahí chaval 👏👏 este año yo cr...,Hope,Generalized Hope
1,"#USER# Si todo bien, yo deseo lo mismo. Dios u...",Hope,Unrealistic Hope
2,#USER# Ojalá llueva con intensidad para paliar...,Hope,Realistic Hope
3,#USER# Dígale ya es vital que los profesores c...,Not Hope,Not Hope
4,#AGRADECIMIENTOS | Finalizados los cultos en ...,Hope,Generalized Hope


In [12]:
dev_esp['multiclass'].value_counts()

multiclass
Not Hope            1958
Generalized Hope    1001
Unrealistic Hope     473
Realistic Hope       405
Sarcasm              251
Name: count, dtype: int64

In [13]:
dev_esp['binary'].value_counts()

binary
Not Hope    2162
Hope        1926
Name: count, dtype: int64

In [11]:
df['text'] = df['text'].str.lower()
df.head()

,text
0,"mi niñaaaaaaa preciosa, sigue como vas por ese..."
1,dea q se busque un hombre entonces porque creo...
2,"#user# ay si me parece que es eso, yo estaba e..."
3,#user# #user# #user# esta encuesta que ficha ...
4,"""anhelo lo etéreo, lo remoto, lo sombrío y lo ..."


In [12]:
string.punctuation
def punctuation_remove(text):
    puncts = string.punctuation
    return text.translate(str.maketrans('','', puncts))
    

In [13]:
df['text'] = df['text'].apply(lambda x: punctuation_remove(x))
df.head()

,text
0,mi niñaaaaaaa preciosa sigue como vas por ese ...
1,dea q se busque un hombre entonces porque creo...
2,user ay si me parece que es eso yo estaba espe...
3,user user user esta encuesta que ficha técnic...
4,anhelo lo etéreo lo remoto lo sombrío y lo dud...


In [14]:
# from nltk.corpus import stopwords
# # stp_wrds = set(stopwords.words('english'))
# # print(stp_wrds)

In [15]:
# new_stopwords = set(stopwords.words('english')) - {'not', 'these', 'doesn', 'wouldn', "shouldn't", "couldn't" "isn't","hadn't",
#                                                   "didn","mustn't", "hasn't", "wasn't", 'but', 'mustn'"weren't", 'couldn',"didn't",
#                                                   "shan't",'no', "mightn't", "wouldn't","aren't", "haven't", 'mightn', 'not','hadn',
#                                                   'don', 'wasn', 'haven', "don't", 'isn', 'needn', 't', 'shouldn', "doesn't","needn't",'shan'}


In [16]:
# def stopwords_removal(text):
#     return " ".join([word for word in text.split() if word not in new_stopwords])

In [17]:
# df['text'] = df['text'].apply(lambda x: stopwords_removal(x))
# df.head()


In [18]:
def remove_emojis(data):
    emoji = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoji, '', data)

In [19]:
df["text"] = df["text"].apply(lambda x:remove_emojis(x))
df.head()

,text
0,mi niñaaaaaaa preciosa sigue como vas por ese ...
1,dea q se busque un hombre entonces porque creo...
2,user ay si me parece que es eso yo estaba espe...
3,user user user esta encuesta que ficha técnic...
4,anhelo lo etéreo lo remoto lo sombrío y lo dud...


In [20]:
def clean_text(text):
    text = re.sub(r'<.*?>', '', text)  
    text = re.sub(r'http\S+|www\S+', '', text)  
    text = re.sub(r'\S+@\S+', '', text)  
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  
    return text


In [21]:
df["text"] = df["text"].apply(clean_text)
df.head()

,text
0,mi niaaaaaaa preciosa sigue como vas por ese c...
1,dea q se busque un hombre entonces porque creo...
2,user ay si me parece que es eso yo estaba espe...
3,user user user esta encuesta que ficha tcnica...
4,anhelo lo etreo lo remoto lo sombro y lo dudos...


In [22]:
def remove_urls(data):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', data)

In [23]:
df["text"] = df["text"].apply(remove_urls)
df.head()

,text
0,mi niaaaaaaa preciosa sigue como vas por ese c...
1,dea q se busque un hombre entonces porque creo...
2,user ay si me parece que es eso yo estaba espe...
3,user user user esta encuesta que ficha tcnica...
4,anhelo lo etreo lo remoto lo sombro y lo dudos...


In [24]:
import re

def clean_noise(data):
    if not isinstance(data, str):  
        return ""
    data = data.lower().strip()  
    data = re.sub(r'\s+', ' ', data)
    data = re.sub(r'\b[a-zA-Z0-9]+gmailcom\b', '', data)
    data = re.sub(r'\bhttp\S+\b', '', data)
    data = re.sub(r'\btextwhatsapp\d+\b', '', data)
    data = re.sub(r'\b[a-zA-Z]*emoticonx\w+\b', '', data)
    data = re.sub(r'\b[a-zA-Z0-9_]{5,}\d{3,}\b', '', data)
    data = re.sub(r'\b(\d{4,}(\s\d{4,})+)\b', '', data)
    data = data.strip()
    return data

In [25]:
df["text"] = df["text"].apply(clean_noise)
df.head()

,text
0,mi niaaaaaaa preciosa sigue como vas por ese c...
1,dea q se busque un hombre entonces porque creo...
2,user ay si me parece que es eso yo estaba espe...
3,user user user esta encuesta que ficha tcnica ...
4,anhelo lo etreo lo remoto lo sombro y lo dudos...


In [26]:
df["text"] = df["text"].apply(clean_noise)
df.head()

,text
0,mi niaaaaaaa preciosa sigue como vas por ese c...
1,dea q se busque un hombre entonces porque creo...
2,user ay si me parece que es eso yo estaba espe...
3,user user user esta encuesta que ficha tcnica ...
4,anhelo lo etreo lo remoto lo sombro y lo dudos...


In [27]:
from collections import Counter
def counter_word(text_col):
    count = Counter()
    for text in text_col.values:
        for word in text.split():
            count[word] += 1
    return count
    
counter = counter_word(df.text)
counter.most_common(10)

[('que', 5515),
 ('de', 5121),
 ('user', 3838),
 ('y', 3436),
 ('la', 3197),
 ('a', 3134),
 ('el', 2617),
 ('en', 2322),
 ('no', 2252),
 ('un', 1431)]

In [28]:

df["text"] = df["text"].str.replace(r"\buser\b", "", regex=True)  #Delete "user"
df["text"] = df["text"].str.replace(r"\burl\b", "", regex=True)   #Delete "url"

df["text"] = df["text"].str.strip()
df.head()


,text
0,mi niaaaaaaa preciosa sigue como vas por ese c...
1,dea q se busque un hombre entonces porque creo...
2,ay si me parece que es eso yo estaba esperando...
3,esta encuesta que ficha tcnica tiene porque do...
4,anhelo lo etreo lo remoto lo sombro y lo dudos...


In [29]:
# from spellchecker import SpellChecker

# spell = SpellChecker()
# def correct_spellings(text):
#     corrected_text = []
#     misspelled_words = spell.unknown(text.split())
#     for word in text.split():
#         if word in misspelled_words:
#             corrected_text.append(spell.correction(word))
#         else:
#             corrected_text.append(word)
#     return " ".join(corrected_text)
        


In [30]:
# df["text"] = df["text"].apply(clean_noise)
# df.head()

In [31]:
# import spacy

# nlp = spacy.load("en_core_web_sm")

# def lemmatize_text(text):
#     doc = nlp(text)
#     return " ".join([token.lemma_ for token in doc])



In [32]:
# df["text"] = df["text"].apply(lemmatize_text)
# df.head()

In [33]:
# num_unique_words = len(counter)
# print(num_unique_words)

In [34]:
# frequent_words = set(word for (word, wc) in counter.most_common(2))
# def remove_freq_words(text):
#     return " ".join([word for word in text.split() if word not in frequent_words])

In [35]:
# df['text'] = df['text'].apply(lambda x: remove_freq_words(x))
# df.head()

In [36]:
# df['binary'] = df['binary'].map({'Hope': 1, 'Not Hope': 0})
# df['multiclass'] = df['multiclass'].map({
#     "Generalized Hope" : 0,
#     "Not Hope" : 1,
#     "Realistic Hope" : 2,
#     "Sarcasm" : 3,
#     "Unrealistic Hope" : 4})

df.to_csv("preprocessed data path", index=False, encoding="utf-8")
